In [1]:
"""setup"""
import json
from unittest import mock as mocker
from mock_server import create_app
import utils
import os
import tempfile
import tensorflow as tf
import wandb
wandb.reset_env()
app = create_app(resume=False)
mock = utils.RequestsMock(app.test_client(), {})
mocker.patch("gql.transport.requests.requests", mock).start()
mocker.patch("wandb.apis.file_stream.requests", mock).start()
mocker.patch("wandb.apis.internal.requests", mock).start()

os.chdir(tempfile.gettempdir())
os.environ[wandb.env.ENTITY] = "mock_server_entity"
os.environ[wandb.env.API_KEY] = "0" * 40
os.environ[wandb.env.BASE_URL] = "http://test"

wandb.init(project="tf2-board", sync_tensorboard=True)

W1118 20:32:54.187056 4717663680 _import_c_extension.py:41] This caffe2 python run does not have GPU support. Will run in CPU only mode.


W&B Run: http://test/mock_server_entity/test/runs/26di9m0z

In [2]:
"""test_summary_writer"""
writer = tf.summary.create_file_writer(".")
with writer.as_default():
    for i in range(10):
        tf.summary.scalar('loss', i, step=i)

In [3]:
parsed = [json.loads(line) for line in open("{}/wandb-history.jsonl".format(wandb.run.dir)).readlines()]
print(parsed[-1])

{'global_step': 9, '_timestamp': 1574113548.313594, 'loss': 9.0, '_runtime': 1.181638240814209, '_step': 9}


In [4]:
"""test_second_init"""
wandb.init(project="tf2-board", sync_tensorboard=True)
wandb.log({"cool": 1})

In [5]:
"""test_bam"""
lines = [json.loads(line) for line in open("{}/wandb-history.jsonl".format(wandb.run.dir)).readlines()]
lines[-1]

{'cool': 1,
 '_runtime': 4.047835826873779,
 '_timestamp': 1574113567.1933959,
 '_step': 0}